<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `CurviIGH_NRPy`: Source Terms

## Author: Terrence Pierre Jacques & Patrick Nelson

<a id='intro'></a>

**Notebook Status:** <font color=green><b> Validated </b></font>

**Validation Notes:** This code produces the expected results for generated functions.

## This module presents the functionality of [GiRaFFE_NRPy_Source_Terms.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Source_Terms.py).

## Introduction: 
This writes and documents the C code that `GiRaFFE_NRPy` uses to compute the source terms for the right-hand sides of the evolution equations for the unstaggered prescription.

The equations themselves are already coded up in other functions; however, for the $\tilde{S}_i$ source term, we will need derivatives of the metric. It will be most efficient and accurate to take them using the interpolated metric values that we will have calculated anyway; however, we will need to write our derivatives in a nonstandard way within NRPy+ in order to take advantage of this, writing our own code for memory access.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#stilde_source): The $\tilde{S}_i$ source term
1. [Step 3](#code_validation): Code Validation against original C code
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1. Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

Here we import some core NRPy+ and python modules.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
GRMHD_dir_path = os.path.join("../GRMHD_formulation/")
sys.path.append(GRMHD_dir_path)
import GRMHD_equations_new_version as GRMHD    # NRPy+: Generate general relativistic magnetohydrodynamics equations

nrpy_dir_path = os.path.join("../nrpy/nrpytutorial/")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 1: The StildeD RHS *source* term
# Step P1: Import needed NRPy+ core modules:
from outputC import outputC, outCfunction, outC_function_dict # NRPy+: Core C code output module
import finite_difference as fin       # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par        # NRPy+: Parameter interface
import grid as gri                    # NRPy+: Functions having to do with numerical grids
import reference_metric as rfm        # NRPy+: Reference metric support
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import cmdline_helper as cmd          # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys                # Standard Python modules for multiplatform OS-level functions
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends

thismodule = "IGM_Source_Terms"

Ccodesdir = "IGM_standalone_Ccodes/"
cmd.mkdir(os.path.join(Ccodesdir))

par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

CoordSystem = "Cartesian"

# Set coordinate system to dst_basis
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

import IGM_All_Source_Terms as ST

<a id='stilde_source'></a>

# Step 2: The $\tilde{S}_i$ source term \[Back to [top](#toc)\]
$$\label{stilde_source}$$


In [2]:
# Generate SymPy symbolic expressions
formalism="BSSN"
GRMHD.set_up_base_vars(formalism)

GRMHD.compute_vU_from_u4U__no_speed_limit(GRMHD.u4U)   

GRMHD.compute_sqrtgammaDET(GRMHD.gammaDD)
GRMHD.compute_smallb4U(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.u4U, GRMHD.BU, GRMHD.sqrt4pi)
GRMHD.compute_smallbsquared(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.smallb4U)

# First compute stress-energy tensor T4UU and T4UD:
GRMHD.compute_T4UU(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.rho_b,GRMHD.P,GRMHD.h,GRMHD.u4U, GRMHD.smallb4U, GRMHD.smallbsquared)
GRMHD.compute_T4UD(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.T4UU)

# Compute conservative variables in terms of primitive variables
GRMHD.compute_rho_star(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.rho_b,GRMHD.u4U)
GRMHD.compute_tau_tilde(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UU,GRMHD.rho_star)
GRMHD.compute_S_tildeD(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

# Next compute fluxes of conservative variables
GRMHD.compute_rho_star_fluxU(GRMHD.VU,GRMHD.rho_star)
GRMHD.compute_tau_tilde_fluxU(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.VU, GRMHD.T4UU, GRMHD.rho_star)
GRMHD.compute_S_tilde_fluxUD (GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

# Then declare derivatives & compute g4DD_zerotimederiv_dD
GRMHD.compute_g4DD_zerotimederiv_dD(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.gammaDD_dD,GRMHD.betaU_dD,GRMHD.alpha_dD)

# Then compute source terms on tau_tilde and S_tilde equations
GRMHD.compute_tau_tilde_source_term(GRMHD.KDD,GRMHD.betaU,GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.alpha_dD, GRMHD.T4UU)
GRMHD.compute_S_tilde_source_termD(GRMHD.alpha,GRMHD.sqrtgammaDET,GRMHD.g4DD_zerotimederiv_dD, GRMHD.T4UU)


Next we write the symbolic expressions to a C file.

In [3]:
tau_tilde_source_term_free_symbols = GRMHD.tau_tilde_source_term.free_symbols
S_tilde_source_termD0_free_symbols = GRMHD.S_tilde_source_termD[0].free_symbols
S_tilde_source_termD1_free_symbols = GRMHD.S_tilde_source_termD[1].free_symbols
S_tilde_source_termD2_free_symbols = GRMHD.S_tilde_source_termD[2].free_symbols

all_free_sysmbols = tau_tilde_source_term_free_symbols.union(\
                             S_tilde_source_termD0_free_symbols, 
                             S_tilde_source_termD1_free_symbols,
                             S_tilde_source_termD2_free_symbols)

prims_velocities = ["u4U0", "u4U1", "u4U2", "u4U3"]
    
prims = prims_velocities + ["BU0", "BU1", "BU2", "P", "h", "rhob"]
params = ["GAMMA_SPEED_LIMIT", "TINYDOUBLE", "sqrt4pi"]

prestring = ""

for var in all_free_sysmbols:
    if str(var) in prims:
        prestring += "const double "+str(var)+" = prims->"+str(var)+";\n"
    if str(var) in params:
        prestring += "const double "+str(var)+" = rhss_params->"+str(var)+";\n"

prestring += "const double "+str(GRMHD.alpha)+" = metric_quantities->"+str(GRMHD.alpha)+";\n"

if formalism=="BSSN":
    prestring += "const double "+str(GRMHD.Bq.trK)+" = metric_quantities->"+str(GRMHD.Bq.trK)+";\n"
    prestring += "const double "+str(GRMHD.Bq.cf)+" = metric_quantities->"+str(GRMHD.Bq.cf)+";\n"

    for i in range(3):
        vetU_var = GRMHD.Bq.vetU[i]
        prestring += "const double "+str(vetU_var)+" = metric_quantities->"+str(vetU_var)+";\n"

    for i in range(3):
        for j in range(3):
            aDD_var = GRMHD.Bq.aDD[i][j]
            prestring += "const double "+str(aDD_var)+" = metric_quantities->"+str(aDD_var)+";\n"

    for i in range(3):
        for j in range(3):
            hDD_var = GRMHD.Bq.hDD[i][j]
            prestring += "const double "+str(hDD_var)+" = metric_quantities->"+str(hDD_var)+";\n"
            
    for var in all_free_sysmbols:
        if "_dD" in str(var):
            prestring += "const double "+str(var)+" = metric_quantities_derivatives->"+str(var)+";\n"

else:
    for i in range(3):
            betaU_var = GRMHD.betaU[i]
            prestring += "const double "+str(betaU_var)+" = metric_quantities->"+str(betaU_var)+";\n"

    for i in range(3):
        for j in range(3):
            KDD_var = GRMHD.KDD[i][j]
            prestring += "const double "+str(KDD_var)+" = metric_quantities->"+str(KDD_var)+";\n"

    for i in range(3):
        for j in range(3):
            gammaDD_var = GRMHD.gammaDD[i][j]
            prestring += "const double "+str(gammaDD_var)+" = metric_quantities->"+str(gammaDD_var)+";\n"
            
    for var in all_free_sysmbols:
        if "_dD" in str(var):
            prestring += "const double "+str(var)+" = metric_quantities_derivatives->"+str(var)+";\n"

In [4]:
outCparams = "outCverbose=False,CSE_sorting=canonical,CSE_enable=True"
desc = "Adds source term and connection terms to Stilde, rho_star and tau_tilde"
#     includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
name = "calculate_all_source_terms"
vars_to_write = ["conservative_sources->StildeD0_rhs", "conservative_sources->StildeD1_rhs", "conservative_sources->StildeD2_rhs", "conservative_rhs->tau_tilde_rhs"]

vars_rhs = [GRMHD.S_tilde_source_termD[0], 
            GRMHD.S_tilde_source_termD[1], 
            GRMHD.S_tilde_source_termD[2], 
            GRMHD.tau_tilde_source_term]

body = outputC(vars_rhs, vars_to_write, params=outCparams, 
               filename="returnstring", prestring=prestring)


c_type = "void"
params   = "const rhss_paramstruct *restrict rhss_params, "
params  += "const prims_struct *restrict prims, "
params  += "const metric_quantities_struct *restrict metric_quantities, "
params  += "const metric_quantities_derivatives_struct *restrict metric_quantities_derivatives, "
params  += "conservative_sources_struct *restrict conservative_sources"

outCfunction(
    outfile=os.path.join(Ccodesdir,name+".h"),
#     includes=includes,
    desc=desc,
    c_type=c_type, name=name, params=params,
    enableCparameters=False,
    body=body)

Output C function calculate_all_source_terms() to file IGM_standalone_Ccodes/calculate_all_source_terms.h


<a id='code_validation'></a>

# Step 3:  Code Validation against `GiRaFFE_NRPy.Stilde_flux` NRPy+ Module \[Back to [top](#toc)\]
$$\label{code_validation}$$


Here, as a code validation check, we verify agreement in the C code generated by 
1. this tutorial and 
2. the NRPy+ [CurviGiRaFFE_NRPy.CurviGiRaFFE_Stilde_Source_Terms](../../edit/in_progress/CurviGiRaFFE_NRPy.CurviGiRaFFE_Stilde_Source_Terms.py) module.

In [5]:
# Define the directory that we wish to validate against:
valdir = "IGM_val/"
cmd.mkdir(os.path.join(valdir))

import IGM_All_Source_Terms as ST
ST.add_to_Cfunction_dict__Stilde_SourceTerms(formalism=formalism)

with open(os.path.join(valdir,name+".h"), "w") as file:
    file.write(outC_function_dict[name])

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare
files = ["calculate_all_source_terms.h"]

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .py file. See differences above.")
            sys.exit(1)

Printing difference between original C code and this code...
Checking file calculate_all_source_terms.h
No difference. TEST PASSED!


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy_C_code_library-Source_Terms](TTutorial-GiRaFFE_NRPy_C_code_library-Source_Terms.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [6]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy-Source_Terms",location_of_template_file=os.path.join(".."))

[NbConvertApp] WARNING | pattern 'Tutorial-GiRaFFE_NRPy-Source_Terms.ipynb' matched no files
Created Tutorial-GiRaFFE_NRPy-Source_Terms.tex, and compiled LaTeX file to
    PDF file Tutorial-GiRaFFE_NRPy-Source_Terms.pdf
